In [ ]:
import analyzer.plotting
import peakquality as pq
import pickle
import analyzer
from analyzer.datasets import SampleManager
from analyzer.core import AnalysisResult
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from analyzer.plotting import PlotObject, drawAs1DHist, drawAs2DHist
from hist import Hist
from scipy.optimize import curve_fit
import warnings

In [ ]:
SIGNAL_NAMES = ['signal_312_1000_400',
                'signal_312_1000_600',
                'signal_312_1000_700',
                'signal_312_1000_800',
                'signal_312_1000_900',
                'signal_312_1200_400',
                'signal_312_1200_600',
                'signal_312_1200_700',
                'signal_312_1200_800',
                'signal_312_1200_900',
                'signal_312_1200_1000',
                'signal_312_1200_1100',
                'signal_312_1300_400',
                'signal_312_1300_600',
                'signal_312_1300_1200',
                'signal_312_1400_400',
                'signal_312_1400_600',
                'signal_312_1400_1300',
                'signal_312_1500_400',
                'signal_312_1500_600',
                'signal_312_1500_900',
                'signal_312_1500_1000',
                'signal_312_1500_1100',
                'signal_312_1500_1200',
                'signal_312_1500_1300',
                'signal_312_1500_1350',
                'signal_312_1500_1400',
                'signal_312_1500_1450',
                'signal_312_2000_400',
                'signal_312_2000_600',
                'signal_312_2000_900',
                'signal_312_2000_1200',
                'signal_312_2000_1300',
                'signal_312_2000_1400',
                'signal_312_2000_1500',
                'signal_312_2000_1600',
                'signal_312_2000_1700',
                'signal_312_2000_1900']

In [ ]:
s = SampleManager()
s.loadSamplesFromDirectory("datasets/")

a = AnalysisResult.fromFile("output_signal_finerbins.pkl")
a_background = AnalysisResult.fromFile("output_background.pkl")
# hists = a.getMergedHistograms(s)
bg_hists = a_background.getMergedHistograms(s)

In [ ]:
def gaussian(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

def iterative_gaussian_fit(hists, xvar, core_width=1.5, tol=0.05):
    hist = hists[xvar]
    all_data, all_bin_edges = hist.to_numpy()
    all_bin_ctrs = (all_bin_edges[:-1] + all_bin_edges[1:]) / 2
    gaussian_xvals = np.linspace(all_bin_ctrs[0], all_bin_ctrs[-1], 1000)
    
    gaussians = []
    coeffs = []

    # gaussian fit to entire range
    initial_coeff_guess = [np.max(all_data), all_bin_ctrs[np.argmax(all_data)], 100]
    coeff, _ = curve_fit(gaussian, all_bin_ctrs, all_data, p0=initial_coeff_guess)
    gaussian_xvals = np.linspace(all_bin_ctrs[0], all_bin_ctrs[-1], 1000)
    gaussians.append([gaussian_xvals, gaussian(gaussian_xvals, *coeff)])
    coeffs.append(coeff)
    
    iters = 0

    _, prev_mu, prev_sigma = initial_coeff_guess
    _, mu, sigma = coeff

    # iterate over smaller and smaller ranges until the tolerance is satisfied
    # essentailly a do-while loop, since we need at least one iteration
    while True:
        iters += 1
        mask = (mu - core_width * sigma < all_bin_ctrs) & (all_bin_ctrs < mu + core_width * sigma)
        masked_data = all_data[mask]
        masked_bin_ctrs = all_bin_ctrs[mask]

        coeff, _ = curve_fit(gaussian, masked_bin_ctrs, masked_data, p0=coeff)
        prev_mu, prev_sigma = mu, sigma
        _, mu, sigma = coeff

        if abs((mu - prev_mu) / prev_mu) < tol and abs((sigma - prev_sigma) / prev_sigma) < tol:
            print(f"final iteration: mu={mu}, prev_mu={prev_mu}, sigma={sigma}. prev_sigma={prev_sigma}")
            break

        if iters > 100:
            warnings.warn(f"Iterative Gaussian fit never converged. The algorithm may be oscillating between two fits that are not within the tolerance.")
            break

    final_mask = (mu - core_width * sigma < gaussian_xvals) & (gaussian_xvals < mu + core_width * sigma)
    return gaussian_xvals, final_mask, coeff

In [ ]:
def gaussian_fit_performance_comparison(control_key, test_key, mass_threshold, create_plots=False):
    performances_over_mass_plane = []
    for signal_name in SIGNAL_NAMES:
        signal_name_parts = signal_name.split('_')
        true_mtop = int(signal_name_parts[-2])
        true_mchi = int(signal_name_parts[-1])
        if true_mchi / true_mtop : continue # clearly nonphysical here

        histograms = a.results[signal_name].getScaledHistograms(s, None)
        control_signal_obj = PlotObject.fromHist(histograms[control_key])
        test_signal_obj = PlotObject.fromHist(histograms[test_key])
        
        gaussian_xvals_C, core_mask_C, coeff_C = iterative_gaussian_fit(a.results[signal_name].getScaledHistograms(s, None), control_key, tol=0.05)
        Aval_C, mu_C, sigma_C = coeff_C

        gaussian_xvals_T, core_mask_T, coeff_T = iterative_gaussian_fit(a.results[signal_name].getScaledHistograms(s, None), test_key, tol=0.05)
        Aval_T, mu_T, sigma_T = coeff_T

        if create_plots:
            fig, ax = plt.subplots(1, 2)
            drawAs1DHist(ax[0], control_signal_obj, fill=False)
            left_mask_C = (~core_mask_C) & (gaussian_xvals_C < mu_C)
            right_mask_C = (~core_mask_C) & (gaussian_xvals_C > mu_C)
            ax[0].plot(gaussian_xvals_C[core_mask_C], gaussian(gaussian_xvals_C[core_mask_C], *coeff_C), linestyle='-', linewidth=2, color='orange', label='Gaussian core')
            ax[0].plot(gaussian_xvals_C[left_mask_C], gaussian(gaussian_xvals_C[left_mask_C], *coeff_C), linestyle=':', linewidth=2, color='orange', label='Gaussian tails')
            ax[0].plot(gaussian_xvals_C[right_mask_C], gaussian(gaussian_xvals_C[right_mask_C], *coeff_C), linestyle=':', linewidth=2, color='orange')

            drawAs1DHist(ax[1], test_signal_obj, fill=False)
            left_mask_T = (~core_mask_T) & (gaussian_xvals_T < mu_T)
            right_mask_T = (~core_mask_T) & (gaussian_xvals_T > mu_T)
            ax[1].plot(gaussian_xvals_T[core_mask_T], gaussian(gaussian_xvals_T[core_mask_T], *coeff_T), linestyle='-', linewidth=2, color='orange', label='Gaussian core')
            ax[1].plot(gaussian_xvals_T[left_mask_T], gaussian(gaussian_xvals_T[left_mask_T], *coeff_T), linestyle=':', linewidth=2, color='orange', label='Gaussian tails')
            ax[1].plot(gaussian_xvals_T[right_mask_T], gaussian(gaussian_xvals_T[right_mask_T], *coeff_T), linestyle=':', linewidth=2, color='orange')

            #TODO: fix this
            # ax.set_title(f"Chargino mass reconstruction with Gaussian fit\nTrue m_t={true_mtop}, true m_chi={true_mchi}, key={key}\nfinal fit width={(gaussian_xvals[final_mask][-1]-gaussian_xvals[final_mask][0]):.2f}, stddev={sigma:.2f}")
            # ax.set_xlabel("Chargino mass (GeV)")
            # ax.set_ylabel("Number of events")
            # max_index = np.argmax(gaussian(gaussian_xvals, *coeff))
            # xlim = 2 * max(true_mchi, gaussian_xvals[max_index])
            # ax.set_xlim([0, xlim])
            # ax.legend()

        # plt.show()
        performances_over_mass_plane.append((true_mtop, true_mchi, sigma_T/sigma_C))